In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib
!pip install 'prompt-toolkit==1.0.15'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-tk is already the newest version (2.7.15~rc1-1).
The following additional packages will be installed:
  libjbig0 liblcms2-2 libprotobuf10 libprotoc10 libtiff5 libwebp6
  libwebpdemux2 libwebpmux3 libxslt1.1 python-bs4 python-chardet
  python-html5lib python-olefile python-pkg-resources python-six
  python-webencodings
Suggested packages:
  liblcms2-utils python-genshi python-lxml-dbg python-lxml-doc python-pil-doc
  python-pil-dbg python-setuptools
The following NEW packages will be installed:
  libjbig0 liblcms2-2 libprotobuf10 libprotoc10 libtiff5 libwebp6
  libwebpdemux2 libwebpmux3 libxslt1.1 protobuf-compiler python-bs4
  python-chardet python-html5lib python-lxml python-olefile python-pil
  python-pkg-resources python-six python-webencodings
0 upgraded, 19 newly installed, 0 to remove and 3 not upgraded.
Need to get 3,740 kB of archives.
After this operation, 14.6 MB of addition

In [3]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 22589 (delta 0), reused 1 (delta 0), pack-reused 22583
Receiving objects: 100% (22589/22589), 560.00 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (13399/13399), done.
Checking out files: 100% (2836/2836), done.


In [4]:
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/

Cloning into 'cocoapi'...
remote: Enumerating objects: 947, done.
remote: Total 947 (delta 0), reused 0 (delta 0), pack-reused 947
Receiving objects: 100% (947/947), 11.69 MiB | 6.25 MiB/s, done.
Resolving deltas: 100% (565/565), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python2.7/dist-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-2.7
creating build/temp.linux-x86_64-2.7/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -Wstrict-prototypes -fno-strict-aliasing -Wdate-time -D_FORTIFY_SOURCE=2 -g -fdebug-prefix-map=/build/python2.7-nbjU53/python2.7-2.7.15~rc1=

In [5]:
import os
os.chdir('models/research')
!mkdir train eval
import sys 
sys.path.append('/content/models/research/slim')
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim
!protoc object_detection/protos/*.proto --python_out=.


env: PYTHONPATH=/content/models/research:/content/models/research/slim


In [0]:
%run object_detection/builders/model_builder_test.py

In [0]:
os.chdir('object_detection/models')
os.mkdir('model')
os.chdir('model')
os.mkdir('train')
os.mkdir('eval')

In [9]:
os.chdir('..')
os.chdir('..')
os.chdir('..')
os.chdir('..')
os.chdir('..')
print(os.getcwd())

/content


In [0]:
#must be in content
!cp clothes_train.record models/research/object_detection/data
#!cp racoon_test.record models/research/object_detection/data
!cp clothes_label_map.pbtxt models/research/object_detection/data

!cp clothes.config models/research/object_detection/models/model
!cp model.ckpt.data-00000-of-00001 models/research/object_detection/models/model
!cp model.ckpt.index models/research/object_detection/models/model
!cp model.ckpt.meta models/research/object_detection/models/model

In [11]:
#navigate back to research
os.chdir('models/research/')
print(os.getcwd())

/content/models/research


In [12]:

!python object_detection/model_main.py \
    --pipeline_config_path='object_detection/models/model/clothes.config' \
    --model_dir='object_detection/models/model/' \
    --num_train_steps=40000 \
    --sample_1_of_n_eval_examples=1 \
    --alsologtostderr

os.mkdir('object_detection/export_dir')

!python object_detection/export_inference_graph.py \
    --input_type='image_tensor' \
    --pipeline_config_path='object_detection/models/model/clothes.config' \
    --trained_checkpoint_prefix='object_detection/models/model/model.ckpt-40000' \
    --output_directory='object_detection/export_dir'

file = drive.CreateFile({'title':'clothes_inference_graph.pb'})
file.SetContentFile('object_detection/export_dir/frozen_inference_graph.pb')
file.Upload()

/content/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "object_detection/model_main.py", line 26, in <module>
    from object_detection import model_lib
  File "/content/models/research/object_detection/model_lib.py", line 27, in <module>
    from object_detection import eval_util
  File "/content/models/research/object_detection/eval_util.py", line 27, in <module>
    from object_detection.metrics import coco_evaluation
  File "/content/models/research/object_detection/metrics/coco_evaluation.py", line 20, in <module>
    from object_detection.metrics import coco_tools
  File "/content/models/research/object_detection/metrics/coco_tools.

IOError: ignored